In [1]:
import os
import pickle
import cv2
import numpy as np
import math
import time
from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot import bgr8_to_jpeg
from jetbot import ObjectDetector
from jetbot import Camera
from jetbot import Robot

### Set up camera and robot, load pre-trained SSD model for COCO

In [2]:
model = ObjectDetector('../Notebooks/object_following/ssd_mobilenet_v2_coco.engine')
camera = Camera.instance(width=300, height=300)
robot = Robot()

In [187]:
# Load COCO labels
filename = "coco_labels.dat"
filehandler = open(filename, 'rb')
COCO_labels = pickle.load(filehandler)

# Load camera calibration data for undistort
filename = "calibration.dat"
filehandler = open(filename, 'rb')
camera_cal = pickle.load(filehandler)
mtx = camera_cal['mtx']
dist = camera_cal['dist']
f_u = mtx[0,0]   # focal center coordinates
f_v = mtx[1,1]
focal_center = np.array([f_u, f_v])

# Mapping between set_motor "speed" and measured wheel angular velocity "omega"
# for 0.1 second motor running time
wheel_calibration = {
    "speed": [0.25, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8],
    "omega": [0.0, 3.85, 9.23, 15.0, 25.8, 29.2, 35.4]
}

# Open Image Widget
image_widget = widgets.Image(format='jpeg', width=300, height=300)
width = int(image_widget.width)
height = int(image_widget.height)

BLUE = (255, 0, 0)
GREEN = (0, 255, 0)
RED = (0, 0, 255)

diag_dir = 'diagnostics'

# we have this "try/except" statement because these next functions can throw an error if the directories exist already
try:
    os.makedirs(diag_dir)
except FileExistsError:
    print('Directories not created becasue they already exist')

Directories not created becasue they already exist


In [96]:
start_bbox_TV = [(102, 90), (161, 134)]

pos1_bbox_TV = [(102, 90), (164, 135)]
pos1_bbox_stool = [(168, 123), (200, 182)]

pos2_bbox_TV = [(99, 79), (170, 128)]
pos2_bbox_stool = [(171, 115), (211, 183)]

pos2left_bbox_TV = [(136, 64), (218, 132)]
pos2left_bbox_stool = [(215, 96), (283, 187)]

pos2right_bbox_TV = [(51, 69), (134, 131)]
pos2right_bbox_stool = [(131, 122), (164, 184)]

pos3_bbox_TV = [(90, 72), (172, 127)]
pos3_bbox_stool = [(172, 107), (219, 189)]

# Bbox for TV and stool at various camera poses
beg_bbox_TV = [(72, 92), (136, 137)]  # Pose (0,0,0)
mid_bbox_TV = [(33, 69), (114, 124)]  # Pose (0.5,0,0)

end_bbox_stool = [(175, 84), (237, 186)]   # Pose (1.0,0,0)

end_bbox_horse = [(147, 103), (247, 201)]

In [176]:
def undistort(img, mtx, dist, crop=False):
    """Undistort camera image based on calibration data"""
    h,w = img.shape[:2]
    # print (h,w)
    newcameramtx, roi=cv2.getOptimalNewCameraMatrix(mtx,dist,(w,h),1,(w,h))
    
    # undistort
    dst = cv2.undistort(img, mtx, dist, None, newcameramtx)

    # crop the image (optional)
    if crop:
        x,y,w,h = roi
        dst = dst[y:y+h, x:x+w]
    return dst

def draw_bbox(img, width, height, bbox, color, line_width):
    bbox_pixel = [(int(width * bbox[0]), int(height * bbox[1])), 
                  (int(width * bbox[2]), int(height * bbox[3]))]
    cv2.rectangle(img, bbox_pixel[0], bbox_pixel[1], color, line_width)
    return bbox_pixel

def display_detected(img, detections, width, height, debug=False):
    """ put blue bounding boxes on detected objects on image """
    
    for det in detections[0]:
        label = COCO_labels[det['label']]
        bbox = det['bbox']
        bbox_pixel = draw_bbox(img, width, height, bbox, BLUE, 1)
        if debug:
            print(label,det['label'], bbox_pixel)
    return

def detection_center(detection):
    """Computes the center x, y coordinates of the object"""
    bbox = detection['bbox']
    center_x = (bbox[0] + bbox[2]) / 2.0 - 0.5
    center_y = (bbox[1] + bbox[3]) / 2.0 - 0.5
    return (center_x, center_y)

def object_center(object_features):
    """Computes the center x, y coordinates of the object using its feature points"""
    UL_fp, UR_fp, LR_fp, LL_fp = obj_features
    UL_fp_x, UL_fp_y = UL_fp
    LR_fp_x, LR_fp_y = LR_fp
    center_x = (UL_fp_x + LR_fp_x) / 2.0 - 0.5
    center_y = (UL_fp_y + LR_fp_y) / 2.0 - 0.5
    return np.array([center_x, center_y])
    
def norm(vec):
    """Computes the length of the 2D vector"""
    return np.sqrt(vec[0]**2 + vec[1]**2)

def closest_detection(detections, debug= False):
    """Finds the detection closest to the image center"""
    closest_detection = None
    for det in detections:
        center = detection_center(det)
        if debug:
            print(center)
        if closest_detection is None:
            closest_detection = det
        elif norm(detection_center(det)) < norm(detection_center(closest_detection)):
            closest_detection = det
    return closest_detection

def create_feature_pts(bbox):
    """ Return 4 feature points from a bounding box """
    # extract bounding x and y coordinates
    UL_fp, LR_fp = bbox
    UL_fp_x, UL_fp_y = UL_fp
    LR_fp_x, LR_fp_y = LR_fp
   
    # return UL(upper left), UR(upper right), LR(lower right), LL(lower left) feature points
    return [(UL_fp_x, UL_fp_y), (LR_fp_x, UL_fp_y), (LR_fp_x, LR_fp_y), (UL_fp_x, LR_fp_y)]

def generate_ref_obj_features(ref_obj, img, detections, width, height, debug=False):
    """ Find reference object in detected objects and generate 4 feature points """
    matching_detections = [d for d in detections[0] if d['label'] == ref_obj]

    # get detection closest to center of field of view and draw it (if model detects multiple
    # reference objects)
    det = closest_detection(matching_detections)

    if det is not None:
        bbox = det['bbox']
        # bound reference object in green
        draw_bbox(img, width, height, bbox, GREEN, 1)
        # convert to pixel units
        bbox_pixel = [(int(width * bbox[0]), int(height * bbox[1])), 
                       (int(width * bbox[2]), int(height * bbox[3]))]
        return create_feature_pts(bbox_pixel) # return 4 feature points of bounding box
    else:
        return None
    
def fp_error(current, goal):
    """ Return error of a feature point """
    current_x, current_y = current
    goal_x, goal_y = goal
    # Use Corke's convention of (p*-p) --> gain is a +ve number
    return (goal_x-current_x, goal_y-current_y)  

def features_error(currentFeatures, goalFeatures):
    """ Return feature points error vector """
    UL_current, UR_current, LR_current, LL_current = currentFeatures 
    UL_goal, UR_goal, LR_goal, LL_goal = goalFeatures 
    
    # calculate error for each feature point
    UL_error = fp_error(UL_current, UL_goal)
    error_vector = np.asarray(UL_error)
    UR_error = fp_error(UR_current, UR_goal)
    error_vector = np.concatenate((error_vector, UR_error), axis=None)
    LR_error = fp_error(LR_current, LR_goal)
    error_vector = np.concatenate((error_vector, LR_error), axis=None)
    LL_error = fp_error(LL_current, LL_goal)
    error_vector = np.concatenate((error_vector, LL_error), axis=None)

    # return 1x8 error vector of UL, UR, LR, LL feature points
    return np.reshape(error_vector, (1,-1))

def image_jacobian(fp, mtx, depth):
    """ Generate image jacobiab L for a feature point """
    # focal lengths in pixel unit
    f_u = mtx[0,0]
    f_v = mtx[1,1]
    c_u = mtx[0,2]
    c_v = mtx[1,2]

    # Estimated distance of reference object (m)
    Z = depth

    # Calculate J of feature point
    u_raw, v_raw = fp
    u = u_raw - c_u
    v = v_raw - c_v    
    
    L = np.array([[-f_u/Z, 0, u/Z, u*v/f_u, -(f_u+u*u/f_u), v],
                  [0, -f_v/Z, v/Z, f_v+v*v/f_v, -u*v/f_v, -u]])
    return L

def robot2world(robot_orientation):
    """ calculate the robot jacobian """
    theta = robot_orientation
    
    return np.array([[math.cos(theta),0],
                  [math.sin(theta),0],
                  [0,0],
                  [0,0],
                  [0,0],
                  [0,1]])

def control2robot(wheel_radius, axle_length):
    """ transform wheel speeds to robot motion in world frame """
    l = axle_length
    r = wheel_radius

    return np.array([[r/2, r/2],
                  [r/l, -r/l]])

def save_snapshot(img, directory, i):
    image_path = os.path.join(directory, 'detect'+str(i+1)+'.jpg')
    cv2.imwrite(image_path, img) 
    return

def omega2speed(in_val, mapping):
    """ Map wheel angular speed to motor speed setting based on a calibration mapping """
    
    if in_val < 0:
        sign = -1
        in_val = abs(in_val)
    else:
        sign = 1
        
    out_lower = 0
    in_lower = 0
    out_val = 0

    for i, in_upper in enumerate(mapping["omega"]):
        # print (i, in_upper)
        if in_val < in_upper:
            out_upper = mapping["speed"][i]
            out_val = out_lower + (in_val - in_lower)/(in_upper - in_lower) \
                *(out_upper-out_lower)
            # print("yes", out_val)
            break
        else:
            # print("no")
            out_lower = mapping["speed"][i]
            in_lower = in_upper
            
    if out_val is 0:
        print ("Input is too high!!!")
        out_val = 0
        
    return sign*out_val

def rotate_left_2wheels(Rtime):
    
    robot.set_motors(-0.3, 0.3)
    time.sleep(Rtime)
    robot.stop()
    
    return

In [172]:
# Display camera image with bounding boxes for detected objects
display(widgets.HBox([image_widget]))

# robot pose - may need to be updated
depth = 2.0
orientation = 0 

# robot parameters
wheel_radius = 0.0325
axle_length = 0.12

# Control loop parameters
gain = 1.4
Rtime = 0.1  # The loop run at 8fps --> 0.125s duration
num_iter = 10
interval = 2

no_motion = True  # Flag to disable motor (for debugging)
debug = True

# Use TV as reference object 
ref_obj = 72
goal_features = create_feature_pts(mid_bbox_TV)
print(ref_obj,COCO_labels[ref_obj], goal_features)


start = time.perf_counter()
# Run for fixed # of iterations
for i in range(num_iter):
    
    image = undistort(camera.value, mtx, dist) # undistort camera image
    detections = model(image) # Use SSD model to detect objects
    display_detected(image, detections, width, height, debug=True) # put bounding boxes on detected objects
    
    # Detect reference object in camera image and place green bounding box around it
    obj_features = generate_ref_obj_features(ref_obj, image, detections, width, height)
    
    if obj_features is None:
        print("Reference object not detected!!!")
        continue
        
    # bound reference object's feature points at goal pose in red
    cv2.rectangle(image, goal_features[0], goal_features[2], RED, 2)
    error = features_error(obj_features,goal_features)
    
    if i%interval == 0 and debug:
        print(obj_features)
        print(error)
    
    if np.linalg.norm(error) < 5.6:
        print ("Goal point achieved!!!")
        continue
        
    T = control2robot(wheel_radius, axle_length) # wheel speeds to robot velocities transform
    H = robot2world(orientation)  # robot velocities to world frame transform
    
    # Image Jacobian based on 4 fp of reference object
    UL_fp, UR_fp, LR_fp, LL_fp = obj_features
    L = np.vstack((image_jacobian(UL_fp, mtx, depth), image_jacobian(UR_fp, mtx, depth), \
              image_jacobian(LR_fp, mtx, depth), image_jacobian(LL_fp, mtx, depth)))
    
    # implement w = gain * pinv(LHT).error
    J = np.dot(L,np.dot(H,T))
    inv_J = np.linalg.pinv(J)
    wheel_velocities = gain * np.dot(np.linalg.pinv(J), error.T)
    robot_velocities = np.dot(T, wheel_velocities)
    w_r = omega2speed(wheel_velocities[0,0],wheel_calibration) 
    w_l = omega2speed(wheel_velocities[1,0],wheel_calibration) 
    
    if i%interval == 0 and debug:
        print("Iteration ", i)
        save_snapshot(image, diag_dir, i)
        print(wheel_velocities, (w_l, w_r))
        print(robot_velocities)
        print("Orientation:", orientation)
        print("Depth:", depth)
    
    if no_motion is False:
        robot.set_motors(w_l, w_r)
        time.sleep(Rtime)
        robot.stop()
    
    # Update robot's orientation
    translation_velocity = robot_velocities[0,0]
    angular_velocity = robot_velocities[1,0]
    orientation += Rtime*angular_velocity
    depth -= Rtime*translation_velocity*math.cos(orientation)
    
    image_widget.value = bgr8_to_jpeg(image)  # update image widget with camera image

image_widget.value = bgr8_to_jpeg(image)
robot.stop()

end = time.perf_counter()
print ("FPS: {:.1f}".format(num_iter/(end-start)))

72 laptop [(33, 69), (114, 69), (114, 124), (33, 124)]
couch 62 [(96, 105), (138, 184)]
clock 84 [(259, 114), (291, 182)]
clock 84 [(246, 118), (281, 181)]
Reference object not detected!!!
couch 62 [(96, 106), (138, 183)]
clock 84 [(259, 113), (290, 183)]
clock 84 [(246, 118), (279, 181)]
Reference object not detected!!!
couch 62 [(96, 105), (139, 183)]
clock 84 [(258, 113), (291, 182)]
clock 84 [(246, 117), (280, 181)]
Reference object not detected!!!
couch 62 [(96, 106), (138, 184)]
clock 84 [(259, 113), (290, 182)]
clock 84 [(245, 118), (280, 181)]
Reference object not detected!!!
couch 62 [(96, 107), (138, 184)]
clock 84 [(258, 114), (290, 182)]
clock 84 [(246, 119), (280, 181)]
Reference object not detected!!!
couch 62 [(96, 103), (138, 184)]
clock 84 [(259, 112), (290, 182)]
clock 84 [(247, 118), (279, 181)]
Reference object not detected!!!
couch 62 [(96, 105), (139, 184)]
clock 84 [(258, 113), (289, 182)]
clock 84 [(247, 117), (280, 180)]
Reference object not detected!!!
couch 6

In [107]:
robot.stop()

In [139]:
# Display camera image with bounding boxes for detected objects
display(widgets.HBox([image_widget]))

# robot pose - may need to be updated
depth = 1.3
orientation = 0 

# Control loop parameters
gain = 1.5
Rtime = 0.1  # The loop run at 8fps --> 0.125s duration
num_iter = 30
interval = 2

no_motion = False  # Flag to disable motor (for debugging)
debug = True

# Use stool as reference object
ref_obj = 62
goal_features = create_feature_pts(end_bbox_stool)
print(ref_obj,COCO_labels[ref_obj], goal_features)

start = time.perf_counter()
# Run for fixed # of iterations
for i in range(num_iter):
    
    image = undistort(camera.value, mtx, dist) # undistort camera image
    detections = model(image) # Use SSD model to detect objects
    display_detected(image, detections, width, height, debug=True) # put bounding boxes on detected objects
    
    # Detect reference object in camera image and place green bounding box around it
    obj_features = generate_ref_obj_features(ref_obj, image, detections, width, height)
    
    if obj_features is None:
        print("Reference object not detected!!!")
        continue
        
    # bound reference object's feature points at goal pose in red
    cv2.rectangle(image, goal_features[0], goal_features[2], RED, 2)
    error = features_error(obj_features,goal_features)
    
    if i%interval == 0 and debug:
        print(obj_features)
        print(error)
    
    if np.linalg.norm(error) < 5.6:
        print ("Goal point achieved!!!")
        continue
        
    T = control2robot(wheel_radius, axle_length) # wheel speeds to robot velocities transform
    H = robot2world(orientation)  # robot velocities to world frame transform
    
    # Image Jacobian based on 4 fp of reference object
    UL_fp, UR_fp, LR_fp, LL_fp = obj_features
    L = np.vstack((image_jacobian(UL_fp, mtx, depth), image_jacobian(UR_fp, mtx, depth), \
              image_jacobian(LR_fp, mtx, depth), image_jacobian(LL_fp, mtx, depth)))
    
    # implement w = gain * pinv(LHT).error
    J = np.dot(L,np.dot(H,T))
    inv_J = np.linalg.pinv(J)
    wheel_velocities = gain * np.dot(np.linalg.pinv(J), error.T)
    robot_velocities = np.dot(T, wheel_velocities)
    w_r = omega2speed(wheel_velocities[0,0],wheel_calibration) 
    w_l = omega2speed(wheel_velocities[1,0],wheel_calibration) 
    
    if i%interval == 0 and debug:
        print("Iteration ", i)
        save_snapshot(image, diag_dir, i)
        print(wheel_velocities, (w_l, w_r))
        print(robot_velocities)
        print("Orientation:", orientation)
        print("Depth:", depth)
    
    if no_motion is False:
        robot.set_motors(w_l, w_r)
        time.sleep(Rtime)
        robot.stop()
    
    # Update robot's orientation
    translation_velocity = robot_velocities[0,0]
    angular_velocity = robot_velocities[1,0]
    orientation += Rtime*angular_velocity
    depth -= Rtime*translation_velocity*math.cos(orientation)
    
    image_widget.value = bgr8_to_jpeg(image)  # update image widget with camera image

image_widget.value = bgr8_to_jpeg(image)
robot.stop()

end = time.perf_counter()
print ("FPS: {:.1f}".format(num_iter/(end-start)))

62 couch [(164, 101), (217, 101), (217, 185), (164, 185)]
couch 62 [(157, 118), (196, 187)]
[(157, 118), (196, 118), (196, 187), (157, 187)]
[[  7 -17  21 -17  21  -2   7  -2]]
Iteration  0
[[-6.30589661]
 [-6.9316925 ]] (-0.35728052972744484, -0.34564863584838446)
[[-0.21511082]
 [ 0.16948639]]
Orientation: 0
Depth: 1.3
couch 62 [(159, 116), (196, 188)]
clock 84 [(98, 151), (106, 181)]
laptop 72 [(27, 59), (107, 118)]
66 65 [(11, 187), (295, 293)]
couch 62 [(162, 118), (201, 186)]
laptop 72 [(28, 63), (115, 120)]
clock 84 [(101, 151), (109, 180)]
[(162, 118), (201, 118), (201, 186), (162, 186)]
[[  2 -17  16 -17  16  -1   2  -1]]
Iteration  2
[[-4.27738139]
 [-4.92782308]] (-0.32003388625250906, -0.307943892012473)
[[-0.14958457]
 [ 0.17616129]]
Orientation: 0.03317503617108727
Depth: 1.3417581131622829
laptop 72 [(42, 64), (124, 122)]
couch 62 [(171, 118), (211, 184)]
couch 62 [(172, 118), (213, 185)]
laptop 72 [(47, 66), (126, 122)]
[(172, 118), (213, 118), (213, 185), (172, 185)]
[

In [188]:
# Use chair as reference object
ref_obj = 62
print(ref_obj,COCO_labels[ref_obj])

# Display camera image with bounding boxes for detected objects
display(widgets.HBox([image_widget]))

Rtime = 0.05

for i in range(50):
    rotate_left_2wheels(Rtime)
    
    image = undistort(camera.value, mtx, dist) # undistort camera image
    detections = model(image) # Use SSD model to detect objects
    display_detected(image, detections, width, height, debug=False) # put bounding boxes on detected objects
    
    # Detect reference object in camera image and place green bounding box around it
    obj_features = generate_ref_obj_features(ref_obj, image, detections, width, height)
    obj_center = object_center(obj_features)
    
    
    image_widget.value = bgr8_to_jpeg(image)  # update image widget with camera image
    
    if obj_features is None:
        print("Reference object not detected!!!")
        continue
    else:
        print(COCO_labels[ref_obj], " at ", obj_features)
        if np.linalg.norm(focal_center - object_center(obj_features)) < 57:
            print("Reference object near focal center !!!")
            break
        continue
 
    time.sleep(1.0)

62 couch


couch  at  [(42, 109), (96, 109), (96, 185), (42, 185)]
couch  at  [(61, 109), (109, 109), (109, 186), (61, 186)]
couch  at  [(76, 108), (122, 108), (122, 185), (76, 185)]
couch  at  [(90, 108), (133, 108), (133, 183), (90, 183)]
Reference object near focal center !!!


## QC

In [131]:
obj_features = [(230, 79), (298, 79), (298, 188), (230, 188)]
error = np.array([[-66,  22, -81,  22, -81,  -3, -66,  -3]])

orientation = 0.16763945741122677
depth = 1.2174696136670464

T = control2robot(wheel_radius, axle_length) # wheel speeds to robot velocities transform
H = robot2world(orientation)  # robot velocities to world frame transform

# Image Jacobian based on 4 fp of reference object
UL_fp, UR_fp, LR_fp, LL_fp = obj_features
L = np.vstack((image_jacobian(UL_fp, mtx, depth), image_jacobian(UR_fp, mtx, depth), \
              image_jacobian(LR_fp, mtx, depth), image_jacobian(LL_fp, mtx, depth)))
    
# implement w = gain * pinv(LHT).error
J = np.dot(L,np.dot(H,T))
inv_J = np.linalg.pinv(J)

wheel_velocities = gain * np.dot(np.linalg.pinv(J), error.T)
robot_velocities = np.dot(T, wheel_velocities)

print (robot_velocities)

[[ 1.0160638 ]
 [-0.25897758]]


In [182]:
np.linalg.norm(np.array([f_u,f_v]) - object_center(obj_features))

102.54403711386931

In [183]:
np.array([f_u,f_v])

array([141.95965107, 183.45309821])

In [184]:
object_center(obj_features)

array([ 46.5, 146. ])

In [185]:
np.array([f_u,f_v])-object_center(obj_features)

array([95.45965107, 37.45309821])

In [186]:
np.linalg.norm(np.array([40,40]))

56.568542494923804